## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
counts=application_df.value_counts("APPLICATION_TYPE")
counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=[]
for i in range(len(counts)):
    if counts[i]<1000: #1000 this time
        application_types_to_replace.append(counts.index[i])

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
countsclass=application_df.value_counts("CLASSIFICATION")
countsclass

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2190        1
C2380        1
C2500        1
C2561        1
C8210        1
Length: 71, dtype: int64

In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace=[]
for i in range(len(countsclass)):
    if countsclass[i]<1000: #1000 this time
        classifications_to_replace.append(countsclass.index[i])
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
application_df.columns

Index(['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS',
       'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [9]:
for i in application_df.columns:
    print(application_df.value_counts(i)) #check value counts for all variables so we can see which would benefit from binning

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
dtype: int64
AFFILIATION
Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
dtype: int64
CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
dtype: int64
USE_CASE
Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
dtype: int64
ORGANIZATION
Trust           23515
Association     10255
Co-operative      486
Corporation        43
dtype: int64
STATUS
1    34294
0        5
dtype: int64
INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
dtype: int64
SPECIAL_CONSIDERATIONS
N    34272
Y       27
dtype: int64

In [10]:
count_aff=application_df.value_counts("AFFILIATION") #bin affiliation
affs_to_replace=[]
for i in range(len(count_aff)):
    if count_aff[i]<100: 
        affs_to_replace.append(count_aff.index[i])
# Replace in dataframe
for cls in affs_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [11]:
count=application_df.value_counts("ORGANIZATION") #bin organization
to_replace=[]
for i in range(len(count)):
    if count[i]<1000: 
        to_replace.append(count.index[i])
# Replace in dataframe
for cls in to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [12]:
count=application_df.value_counts("USE_CASE") #bin use case
to_replace=[]
for i in range(len(count)):
    if count[i]<1000: 
        to_replace.append(count.index[i])
# Replace in dataframe
for cls in to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [13]:
count=application_df.value_counts("INCOME_AMT") #bin income amount
to_replace=[]
for i in range(len(count)):
    if count[i]<1000: 
        to_replace.append(count.index[i])
# Replace in dataframe
for cls in to_replace:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
Other             2790
Name: INCOME_AMT, dtype: int64

In [14]:
quartiles = np.quantile(application_df["ASK_AMT"],[.25,.75]) #removing outliers in the ask amount column
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)
print(lower_bound)
print(upper_bound)
application_df=application_df.loc[(application_df["ASK_AMT"] < upper_bound) & (application_df["ASK_AMT"] > lower_bound)]

887.0
11855.0


In [15]:
reduce_df=application_df.drop(columns=["STATUS","SPECIAL_CONSIDERATIONS"]) #Both were almost entirely one value

In [16]:
dummy_df=pd.get_dummies(reduce_df)#get dummies
print(len(dummy_df))
dummy_df.head()

26093


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,AFFILIATION_Independent,...,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,5000,1,1,0,0,0,0,0,0,1,...,0,0,1,1,0,0,1,0,0,0
2,5000,0,0,0,0,0,1,0,1,0,...,0,0,1,1,0,0,1,0,0,0
3,6692,1,0,0,1,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,1
5,5000,1,0,0,1,0,0,0,0,1,...,0,1,0,0,0,1,1,0,0,0
9,5000,0,0,0,0,0,1,0,1,0,...,0,0,1,1,0,0,1,0,0,0


In [17]:
# Split our preprocessed data into our features and target arrays
y=dummy_df["IS_SUCCESSFUL"]
X=dummy_df.drop(columns=["IS_SUCCESSFUL"])
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input=len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

#0th layer
nn.add(tf.keras.layers.Dense(units=10, input_dim=num_input, activation="relu"))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=10, input_dim=num_input, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, input_dim=num_input, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=10, input_dim=num_input, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=10, input_dim=num_input, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                270       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 10)                110       
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 721 (2.82 KB)
Trainable params: 721 (2.82

In [20]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200) #increased epochs

Epoch 1/200


612/612 [==============================] - 2s 1ms/step - loss: 0.9619 - accuracy: 0.5861
Epoch 2/200
612/612 [==============================] - 1s 1ms/step - loss: 0.6084 - accuracy: 0.7227
Epoch 3/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5779 - accuracy: 0.7319
Epoch 4/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5912 - accuracy: 0.7257
Epoch 5/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5836 - accuracy: 0.7347
Epoch 6/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5913 - accuracy: 0.7295
Epoch 7/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5754 - accuracy: 0.7353
Epoch 8/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5686 - accuracy: 0.7394
Epoch 9/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5706 - accuracy: 0.7410
Epoch 10/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5648 - accu

612/612 [==============================] - 1s 1ms/step - loss: 0.5346 - accuracy: 0.7495
Epoch 156/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5348 - accuracy: 0.7476
Epoch 157/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5338 - accuracy: 0.7485
Epoch 158/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5351 - accuracy: 0.7492
Epoch 159/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7484
Epoch 160/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5438 - accuracy: 0.7477
Epoch 161/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5435 - accuracy: 0.7482
Epoch 162/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5419 - accuracy: 0.7468
Epoch 163/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5370 - accuracy: 0.7485
Epoch 164/200
612/612 [==============================] - 1s 1ms/step - loss: 0.5342 - a

In [22]:
# Evaluate the optomized model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

204/204 - 0s - loss: 0.5568 - accuracy: 0.7557 - 296ms/epoch - 1ms/step
Loss: 0.5568483471870422, Accuracy: 0.7556713819503784


In [23]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5", save_format='h5')

C:\Users\boers\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
